In [1]:
import ruamel.yaml as yaml

import numpy as np
import pandas as pd
from manubrain.storage.postgres import PostgresDataStorage

In [2]:
sql_config = ""
broker_url = ""
broker_port = 0

In [3]:
from sqlalchemy import create_engine
engine = create_engine(sql_config)

In [4]:
import paho.mqtt.client as mqtt

from datetime import datetime
from pathlib import Path

import json


output = 0
ts = datetime.now().strftime("%y-%m-%d-%H:%M:%S-%f")
logdir = "/share/export/logs/"

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("connected OK Returned code=", rc)
    else:
        print("Bad connection Returned code=", rc)
        
def on_message_log(client, userdata, message):
    print(["message [topic, qos, retain flag]=[", message.topic, ", "])
    print([str(message.qos), ", "])
    print([str(message.retain), "]\n"])
    
    if (str(message.topic) == "i/cam"):
        print("message received=((ignore cam))\n")
    else:
        try: 
            print(
                ["message received=" , str(message.payload.decode("utf-8")), "\n\n"]
            )
        except UnicodeDecodeError:
            print(["UnicodeDecodeError: Cannot display message!\n\n"])
            
def print_msg(client, userdata, message):
    m = message.payload.decode("utf-8") 
    j = json.loads(m)
    df = pd.DataFrame(np.zeros((1, 9)))
    print(j)
    for i in range(8):
        df[i] = j["input" + str(i)]
        df = df.rename(columns={i : "value_" + str(i)})
    df = df.rename(columns = {8 : "timestamp"})
    df["timestamp"][0] = pd.to_datetime(j["ts"])
    df.to_sql('fh_sld', engine, if_exists="append")

def on_log(client, userdata, level, buf):
    pass
    # print("log: ", buf)

def create_client_log():
    client = mqtt.Client("", True, None, mqtt.MQTTv31)
    client.on_connect = on_connect  #bind call back function
    client.on_log = on_log
    client.on_message = print_msg  #attach function to callback
    return client

In [5]:
import time
import paho.mqtt.client as mqtt



def sub(t=20):
    client = create_client_log()

    client.connect(broker_url, broker_port)

    client.loop_start()  # start the loop

    # subscribe to everything
    # client.subscribe("#")

    # subscribe to example channels
    # REMARK: its better practice to subscribe in the on_connect method, because otherwise subscriptions are lost when dis- and reconnecting
    client.subscribe("fh/input/sld")
    time.sleep(t)  # wait

    client.loop_stop()  # stop the loop
    client.disconnect()  # disconnect

In [6]:
sub()

connected OK Returned code= 0
{'input0': 1, 'input1': 1619, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:02.005Z'}


<ipython-input-4-5934f5afb286>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["timestamp"][0] = pd.to_datetime(j["ts"])


{'input0': 1, 'input1': 1619, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:03.004Z'}
{'input0': 1, 'input1': 1624, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:04.024Z'}
{'input0': 1, 'input1': 1617, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:05.004Z'}
{'input0': 1, 'input1': 1617, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:06.002Z'}
{'input0': 1, 'input1': 1619, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:07.030Z'}
{'input0': 1, 'input1': 1619, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:08.009Z'}
{'input0': 1, 'input1': 1615, 'input2': 1, 'input3': 0, 'input4': 0, 'input5': 1, 'input6': 1, 'input7': 1, 'ts': '2021-12-07T13:11:09.006Z'}
{'inpu

In [7]:
new_df = pd.read_sql('fh_sld', engine)
print(new_df)

    index  value_0  value_1  value_2  value_3  value_4  value_5  value_6  \
0       0        1     1624        1        0        0        1        1   
1       0        1     1624        1        0        0        1        1   
2       0        1     1624        1        0        0        1        1   
3       0        1     1624        1        0        0        1        1   
4       0        1     1624        1        0        0        1        1   
5       0        1     1624        1        0        0        1        1   
6       0        1     1680        1        0        0        1        1   
7       0        1     1624        1        0        0        1        1   
8       0        1     1624        1        0        0        1        1   
9       0        1     1626        1        0        0        1        1   
10      0        1     1624        1        0        0        1        1   
11      0        1     1626        1        0        0        1        1   
12      0   